In [2]:
!pip install tensorflow


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.4/390.3 MB 11.2 MB/s eta 0:00:35
   ---------------------------------------- 0.7/390.3 MB 11.1 MB/s eta 0:00:36
   ---------------------------------------- 1.9/390.3 MB 15.0 MB/s eta 0:00:26
   ---------------------------------------- 2.6/390.3 MB 15.3 MB/s eta 0:00:26
   ---------------------------------------- 3.8/390.3 MB 17.5 MB/s eta 0:00:23
   ---------------------------------------- 4.6/390.3 MB 18.2 MB/s eta 0:00:22
    --------------------------------------- 5.9/390.3 MB 18.7 MB/s eta 0:00:21
    --------------------------------------- 7.1/390.3 MB 19.7 MB/s eta 0:00:20
    --------------------------------------- 8.1/390.3 MB 19.9 MB/s eta 0:00:20
    --------------------------------------- 9.1/390.3 MB 20.2 MB/s

In [5]:
!pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 325.1 kB/s eta 0:02:00
   ---------------------------------------- 0.1/38.8 MB 544.7 kB/s eta 0:01:12
   ---------------------------------------- 0.1/38.8 MB 544.7 kB/s eta 0:01:12
   ---------------------------------------- 0.1/38.8 MB 544.7 kB/s eta 0:01:12
   ---------------------------------------- 0.1/38.8 MB 544.7 kB/s eta 0:01:12
   ---------------------------------------- 0.1/38.8 MB 423.5 kB/s eta 0:01:32
   ---------------------------------------- 0.1/38.8 MB 423.5 kB/s eta 0:01:32
   ---------------------------------------- 0.2/38.8 MB 518.8 kB/s eta 0:01:15
   ---------------------------------------- 0.2/38.8 MB 518.8 kB/s eta 0:01:15
   ---------------------------------------- 0.3/38.8 MB 548.9 kB/s eta 0:01:11


In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Paths36
train_data_path = "dataset/train"  # Path to training data
model_save_path = "model/traffic_sign_model.h5"  # Path to save the model

# Example mapping for class IDs to labels (update as needed)
class_id_to_name = {
    0: "Speed Limit 20",
    1: "Speed Limit 30",
    2: "Speed Limit 50",
    3: "Speed Limit 60",
    4: "Speed Limit 70",
    5: "Speed Limit 80",
    6: "End of Speed Limit 80",
    7: "Speed Limit 100",
    8: "Speed Limit 120",
    9: "No Overtaking",
    10: "No Overtaking Trucks",
    11: "Right of Way",
    12: "Priority Road",
    13: "Yield",
    14: "Stop",
    15: "No Vehicles",
    16: "No Trucks",
    17: "No Entry",
    18: "General Caution",
    19: "Dangerous Curve Left",
    20: "Dangerous Curve Right",
    21: "Double Curve",
    22: "Bumpy Road",
    23: "Slippery Road",
    24: "Road Narrows",
    25: "Construction",
    26: "Traffic Signals",
    27: "Pedestrians",
    28: "Children Crossing",
    29: "Bicycles Crossing",
    30: "Snow or Ice",
    31: "Wild Animals Crossing",
    32: "End of Restrictions",
    33: "Turn Right Ahead",
    34: "Turn Left Ahead",
    35: "Ahead Only",
    36: "Go Straight or Right",
    37: "Go Straight or Left",
    38: "Keep Right",
    39: "Keep Left",
    40: "Roundabout Mandatory",
    41: "End of No Overtaking",
    42: "End of No Overtaking Trucks",
}


In [21]:
# Load dataset
def load_data(data_path):
    images, labels = [], []
    for label in os.listdir(data_path):
        label_path = os.path.join(data_path, label)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (32, 32))  # Resize to 32x32
                images.append(img)
                labels.append(int(label))  # Convert folder name (label) to integer
    return np.array(images), np.array(labels)

print("Loading data...")
X, y = load_data(train_data_path)
X = X / 255.0  # Normalize pixel values
y = to_categorical(y)  # One-hot encode labels
print(f"Data loaded. Total samples: {X.shape[0]}")


Loading data...
Data loaded. Total samples: 39209


In [22]:
# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y.shape[1], activation='softmax')  # Output layer with softmax activation
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 43)             │         5,547 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,979 (1.22 MB)

 Trainable params: 319,979 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Train the model
print("Training the model...")
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Save the model
if not os.path.exists("model"):
    os.makedirs("model")
model.save(model_save_path)
print(f"Model saved to {model_save_path}")


Training the model...
Epoch 1/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.4843 - loss: 1.9647 - val_accuracy: 0.9310 - val_loss: 0.2484
Epoch 2/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9474 - loss: 0.1948 - val_accuracy: 0.9674 - val_loss: 0.1155
Epoch 3/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9773 - loss: 0.0923 - val_accuracy: 0.9777 - val_loss: 0.0892
Epoch 4/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9853 - loss: 0.0568 - val_accuracy: 0.9839 - val_loss: 0.0649
Epoch 5/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9903 - loss: 0.0362 - val_accuracy: 0.9813 - val_loss: 0.0711
Epoch 6/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9930 - loss: 0.0253 - val_accuracy: 0.9892 - val_loss: 0.0490
Epoch 7/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9944 - loss: 0.0195 - val_accuracy: 0.9907 - val_loss: 0.0453
Epoch 8/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9963 

Model saved to model/traffic_sign_model.h5


In [28]:
# Load the trained model
model = load_model(model_save_path)

# Function to preprocess an image
def preprocess_image(image_path):
    """
    Preprocess the input image for the model.
    :param image_path: Path to the image
    :return: Preprocessed image ready for prediction
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Invalid image file.")
    image = cv2.resize(image, (32, 32))  # Resize to 32x32
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function to predict the class of a traffic sign
def predict_sign(image_path):
    """
    Predict the traffic sign class for the given image.
    :param image_path: Path to the image
    :return: Predicted class name
    """
    try:
        image = preprocess_image(image_path)
        predictions = model.predict(image)
        class_idx = np.argmax(predictions)
        class_name = class_id_to_name.get(class_idx, "Unknown Sign")
        return class_name
    except Exception as e:
        return f"Error: {str(e)}"

# Test the prediction function
test_image_path = "dataset/test/00007.png"  # Replace with your test image path
predicted_sign = predict_sign(test_image_path)
print(f"Predicted Traffic Sign: {predicted_sign}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Traffic Sign: Priority Road
